# Examples for Homework 5.1: Neural Network for Cook Islands Māori Parts of Speech
Dartmouth College, LING48, Spring 2023<br>
Rolando Coto-Solano (Rolando.A.Coto.Solano@dartmouth.edu)

Code modified from:
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

You need to perform three tasks:

(a) Run the program three times and record the training and test accuracy for each of the three runs. What is the average training accuracy? What is the average accuracy for the test set? How are the F1-scores behaving? (You can see more information how the predictions are working by looking at the predictions for the first fifteen items).

(b)	Change the program so that the hidden layers have 48 and 24 neurons, and the output has 3 neurons. Run the modified program three times. What is the average training/test accuracy? How are the F1-scores behaving? (You can see more information how the predictions are working by looking at the predictions for the first ten items).

(c)	Use the same settings as you did in step ‘B’ above and make one more change: Change the program so that it runs for 200 epochs. Run this three times. What is the average training and test accuracy? How are the F1-scores behaving? (You can see more information how the predictions are working by looking at the predictions for the first ten items).

Write all of these answers in a PDF/Word/LibreOffice file. Include screeshots of your results.

In [ ]:
# load packages (neural network is in sklearn)
from numpy import loadtxt
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
import gdown
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

I recommend that you comment out to cell below and then go to<br>
"Runtime" > "Run all". This will automatically run everything,<br>
so you can just record the relevant data and run again and again.

In [ ]:
# Load file and split into training data (nX) and
# the labels we are trying to predict (ny)

cimData = pd.read_csv("cim-3pos.csv")
dataset = cimData
nX = cimData.drop('tokenPOS', axis=1)
ny = cimData['tokenPOS']

numberValidPOS = 3  # noun, verb, preposition

In [ ]:
# Encode the words into numbers and then split
# the randomly data into training and test sets.

encoderX = preprocessing.OneHotEncoder(sparse=True)
X = encoderX.fit_transform(nX)
encoderY = preprocessing.LabelEncoder()
y = encoderY.fit_transform(ny)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10)

inputDims = 1497 # the total of features in the OneHotEncoded X vector (total number of unique words)

In [ ]:
# Here, you can study what the dataset looks like (as text and encoded)

print("--- Training data, predictive features, first row ---")
print(encoderX.inverse_transform(X_train[0:1]))
print("\n--- Training data, predicted result, first row ---")
print(encoderY.inverse_transform(y_train[0:1]))
print("\n--- Training data, predictive features, first row, one-hot encoded ---")
print(X_train[0:1])
print("\n--- Training data, predicted result, first row, one-hot encoded ---")
print(y_train[0:1])

In [ ]:
# define the keras model

model = Sequential()
model.add(Dense(12, input_dim=inputDims, activation='relu'))
model.add(Dense(6, input_dim=inputDims, activation='relu'))
model.add(Dense(1, activation='relu'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

In [ ]:
# make class predictions with the model

print("===== Results from final layer of the first 5 items in test set =====")
t = model.predict(X_test)
print("1: " + str(t[0]))
print("2: " + str(t[1]))
print("3: " + str(t[2]))
print("4: " + str(t[3]))
print("5: " + str(t[4]))

# calculate accuracy
totalCorrect = 0
predictions = []
for i in range(0,len(list(y_test))): 
  tempPred = np.argmax(model.predict(X_test[i], verbose=0), axis=None)
  if (tempPred < numberValidPOS ): predictions.append(tempPred)
  else: predictions.append(0)  # It is possible for the NN to give predictions that are outside of the realm of the labelEncoder. This catches them.
for i in range(0,len(list(y_test))): 
  if (predictions[i] == y_test[i]):
    totalCorrect += 1
accuracy = round((totalCorrect / len(list(y_test)))*100,0)

print("\n===== Size of test set =====")
print(len(list(y_test)))
print("\n===== Test data, predictive features, first 15 rows =====")
print(encoderX.inverse_transform(X_test[0:15]))
print("\n===== Test data, predicted result, first 15 rows =====")
print(predictions[0:15])
print("\n===== Test data, expected result, first 15 rows =====")
print(encoderY.inverse_transform(y_test[0:15]))
print("\n===== Accuracy of test set =====")
print(str(accuracy) + "%")
print("\n===== Predictions =====")


isItCorrect = ""

for i in range(15):
  
  if (i < 9): itemNum = "0" + str(i+1) 
  else: itemNum = str(i+1)
  
  if (predictions[i] == y_test[i]): isItCorrect = "*Correct!*"
  else: isItCorrect = ""
  
  print("item " + itemNum + ": Predicted: " + str(predictions[i]) + " /   " + str(encoderY.inverse_transform([predictions[i]])) + "  \tActual value: " + str(y_test[i]) + " / " + str(encoderY.inverse_transform([y_test[i]])) + " \t" + str(isItCorrect)  )

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(confusion_matrix(y_test, predictions))